# **This notebook's process**

1. Load in Crunchbase dataframes(4 merged CSVs created in `1_SS_EDA.ipynb`)
    - Organizations: `files/output/organizations_merged.csv`
    - Jobs: `files/output/p1_jobs.csv`
    - Investments: `files/output/p1_investments.csv`
    - Partner investments: `files/output/p1_investments_partner.csv`
2. Select date and filter the dataframes by date
3. Save filtered dataframes as separate CSVs, and then load in as SFrames.
    - Crunchbase network: `files/output/graph_temp/cb/{}_df.csv`
    - Pledge 1% network: `files/output/graph_temp/p1/{}_df.csv`
    - Model network: `files/output/graph_temp/model/{}_df.csv`
    - Not Pledge 1% network: `files/output/graph_temp/np1/{}_df.csv`
4. Load SFrames into graph and remove duplicate edges. Produce 8 graphs based on # of edges allowed & direction.
5. Reduce size of dataset by limiting degrees of freedom from Pledge 1% companies, and save the vertices list for a few different network sizes
6. Produce 100 samples of the Crunchbase graphs and save to CSV.
    - 5 Degrees from Pledge 1% Companies: `Model_DF_D5`
        - Baseline: `files/output/Model_DF_D5/B/{}.csv`
        - Baseline Reduced: `files/output/Model_DF_D5/BR/{}.csv`
        - Graph & Baseline: `files/output/Model_DF_D5/GB/{}.csv`
        - Graph & Baseline Reduced: `files/output/Model_DF_D5/GBR/{}.csv`
        - Graph: `files/output/Model_DF_D5/G/{}.csv`
    - 4 Degrees from Pledge 1% Companies: `Model_DF_D4`
        - Baseline: `files/output/Model_DF_D4/B/{}.csv`
        - Baseline Reduced: `files/output/Model_DF_D4/BR/{}.csv`
        - Graph & Baseline: `files/output/Model_DF_D4/GB/{}.csv`
        - Graph & Baseline Reduced: `files/output/Model_DF_D4/GBR/{}.csv`
        - Graph: `files/output/Model_DF_D4/G/{}.csv`

## **Model**
`p1_tag` ~ `rank` + `total_funding_usd` + `age_yr` + `employee_count` (ordinal) + `country` (nominal, 112 indicator columns) + `category_groups` (nominal, 46 indicator columns) + ((GRAPH FEATURES))

In [1]:
'''Importing basic data analysis packages'''
import numpy as np
import pandas as pd
import csv
import warnings
import os
import time
import math
from functools import reduce
from datetime import datetime
warnings.filterwarnings('ignore')

'''Graph'''
import networkx as nx
from pyvis.network import Network
import turicreate
from turicreate import pagerank, kcore, degree_counting, shortest_path, connected_components, triangle_counting
from turicreate import SFrame, SGraph, SArray, load_sgraph, aggregate 

'''Plotting packages'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', font_scale=1.3)

def reduce_mem_usage(df, verbose=True):   
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def network_by_date(date, df_input, jobs_input, invest_input, invest_prtnr_input, model_uuids=[], skip_not_p1=True):
    '''
    This function filters down Crunchbase dataframes by date 
    to ensure that the companies/people/investments being used in modeling exist at a given time.

    INPUT:
        - `date`: string w/ format 'YEAR-MO-DY' (e.g. '2020-09-08')
        - `df`: pandas dataframe of Crunchbase organizationss with necessary column fields:
            * `p1_date`, `founded_on`, `closed_on`
        - `jobs`: pandas dataframe of Crunchbase jobss with necessary column fields:
            * `p1_date`, `started_on`, `ended_on`
        - `invest`: pandas dataframe of Crunchbase investmentss with necessary column fields:
            * `p1_date`, `announced_on`
        - `invest_prtnr`: pandas dataframe of Crunchbase investments with necessary column fields:
            * `p1_date`, `announced_on`
        - `model_uuids`: list that contains the uuids of organizations that are used to construct the model graph
        - `skip_no_p1`: Boolean that defaults to excluding the opposite of the Pledge 1% neighborhood. Likely will delete option altogether later.
    
    OUTPUT:
        - List of dataframe lists, 2 lists of length 12: 
            * [Crunchbase neighborhood dataframes], [Pledge 1% neighborhood dataframes]
                                        OR
              [Crunchbase neighborhood dataframes], [Model neighborhood dataframes]
        - Each dataframe list contains 12 dataframes that will be saved & loaded as SFrames in the next processing step.
            0. Companies
            1. Investors
            2. Investments
            3. Partner investments
            4. Current Jobs
            5. Former jobs
            6. Former affiliated's new jobs
            7. Partner investor's affiliation (if not in jobs dataframes)
            8. Partner investor's coworkers at the investing firm
            9. Partner investor's coworkers' partner investments
            10. Current affiliated's old jobs
            11. Organization nodes from edges in 2,3,6,7,9,10 if not already in 0 or 1
    '''
    # Soft copy of dataframes
    df = df_input.copy()
    jobs = jobs_input.copy()
    invest = invest_input.copy()
    invest_prtnr = invest_prtnr_input.copy()
    
    #*******************************************************************************************************
    # DATE PROCESSING
    
    # Convert date columns to datetime
    df['p1_date'] = pd.to_datetime(df['p1_date'], errors='coerce')
    df['founded_on'] = pd.to_datetime(df['founded_on'], errors='coerce')
    df['closed_on'] = pd.to_datetime(df['closed_on'], errors='coerce')
    jobs['p1_date'] = pd.to_datetime(jobs['p1_date'], errors='coerce')
    jobs['started_on'] = pd.to_datetime(jobs['started_on'], errors='coerce')
    jobs['ended_on'] = pd.to_datetime(jobs['ended_on'], errors='coerce')
    invest['p1_date'] = pd.to_datetime(invest['p1_date'], errors='coerce')
    invest['announced_on'] = pd.to_datetime(invest['announced_on'], errors='coerce')
    invest_prtnr['p1_date'] = pd.to_datetime(invest_prtnr['p1_date'], errors='coerce')
    invest_prtnr['announced_on'] = pd.to_datetime(invest_prtnr['announced_on'], errors='coerce')
    
    # Convert input date to datetime object
    date = pd.Timestamp(date)
    print('\nAS OF {}:\n'.format(date.strftime('%B %d, %Y').upper()))
    
    #*******************************************************************************************************
    # Create new row for tagging model companies
    df['add_to_model'] = 0
    df['add_to_model'][df['uuid'].isin(model_uuids)] = 1
    jobs['add_to_model'] = 0
    jobs['add_to_model'][jobs['org_uuid'].isin(model_uuids)] = 1
    invest['add_to_model'] = 0
    invest['add_to_model'][invest['org_uuid'].isin(model_uuids)] = 1
    invest_prtnr['add_to_model'] = 0
    invest_prtnr['add_to_model'][invest_prtnr['org_uuid'].isin(model_uuids)] = 1
    
    #*******************************************************************************************************
    # COMPANY FILTER
    # Crunchbase company must be founded after DATE and closed before DATE (or DATE == NaT)
    CB_companies = df[(df['founded_on']<=date) & 
                      ((df['closed_on']>date) | (pd.isnull(df['closed_on']))) & 
                      (df['primary_role']=='company')].reset_index(drop=True)
    
    #*******************************************************************************************************
    # INVESTOR FILTER:
    # Crunchbase investor must be founded AFTER date and closed BEFORE date (or date == NaT)
    CB_investors = df[(df['founded_on']<=date) & 
                      ((df['closed_on']>date) | (pd.isnull(df['closed_on']))) & 
                      (df['primary_role']=='investor')].reset_index(drop=True)
    
    #*******************************************************************************************************
    # INVESTMENT FILTER
    # Crunchbase investment must have taken place BEFORE date
    CB_investments = invest[(invest['announced_on']<=date) & 
                            (invest['investor_type']=='organization')].reset_index(drop=True)
    
    #*******************************************************************************************************
    # PARTNER INVESTMENT FILTER
    # Crunchbase partner investment must have taken place BEFORE date
    CB_investment_partners = invest_prtnr[invest_prtnr['announced_on']<=date].reset_index(drop=True)
    
    #*******************************************************************************************************
    # CURRENT JOB FILTER
    # Crunchbase job must have started BEFORE date and ended AFTER date (or date == NaT)
    CB_jobs = jobs[(jobs['job_type'].isin(['executive','board_member','advisor','board_observer'])) & 
                      (jobs['started_on']<=date) & 
                      ((jobs['ended_on']>date) | (pd.isnull(jobs['ended_on'])))].reset_index(drop=True)
    
    #*******************************************************************************************************
    # FORMER JOB FILTER
    # Crunchbase job must have ended BEFORE date or started AFTER date
    CB_jobs_former = jobs[(jobs['job_type'].isin(['executive','board_member','advisor','board_observer'])) & 
                          ((jobs['ended_on']<=date) | (jobs['started_on']>date))].reset_index(drop=True)
    
    #*******************************************************************************************************
    # COMBINE THESE 6 (or 7) INTO LIST OF FRAMES
    lst_of_frames = []
    # Crunchbase frames
    CB_frames = [CB_companies,CB_investors,CB_investments,CB_investment_partners,CB_jobs,CB_jobs_former]
    # Add to list of frames
    lst_of_frames.append(CB_frames)
    # If model_uuids are not supplied, calculate Pledge 1% neighborhood
    if model_uuids == []:
        P1_frames = []
        for frame in CB_frames:
            # Pledge 1% frames must have Crunchbase assumptions in addition to an earlier pledge date
            new_frame = frame[frame['p1_date']<=date].reset_index(drop=True).drop('add_to_model',axis=1)
            P1_frames.append(new_frame)
        # Add to list of frames
        lst_of_frames.append(P1_frames)
    # If model_uuids are supplied, calculate model neighborhood
    if model_uuids != []:
        model_frames = []
        for frame in CB_frames:
            # Include model dataframe if condition satisfied: either are a Pledge 1% company or tagged by model_uuids
            new_frame=frame[(frame['p1_date']<=date) | (frame['add_to_model']==1)].reset_index(drop=True).drop('add_to_model',axis=1)
            model_frames.append(new_frame)
        # Add to list of frames
        lst_of_frames.append(model_frames)
    # If this boolean value is False, calculate ~Pledge 1% neighborhood
    if skip_not_p1 is False:
        not_P1_frames = []
        for frame in CB_frames:
            # Non-Pledge 1% frames must have Crunchbase assumptions in addition to NaT pledge date or later pledge date
            new_frame = frame[(pd.isnull(frame['p1_date']) | (frame['p1_date']>date))].reset_index(drop=True).drop('add_to_model',axis=1)
            not_P1_frames.append(new_frame)
        # Add to list of frames
        lst_of_frames.append(not_P1_frames) 
    # Remove extra column 'add_to_model'
    for idx,frame in enumerate(CB_frames):
        CB_frames[idx] = frame.drop('add_to_model',axis=1)

    #*******************************************************************************************************
    # FORMER NEW JOB FILTER
    print('CaLcUlAtInG... FORMER NEW JOB FILTER')
    
    for frame in lst_of_frames:
        # Where do the former affiliated work now?
        # Pull their uuids
        former_people = frame[5].person_uuid.unique()
        # Pull their current jobs from Crunchbase
        jobs_former_new = CB_frames[4][CB_frames[4].person_uuid.isin(former_people)] 
        # Check they're not already in the current jobs dataframe
        # Combine into one temp data frame
        combined_jobs = pd.concat([frame[4], jobs_former_new]).reset_index(drop=True) 
        df_gpby = combined_jobs.groupby(list(combined_jobs.columns))
        # Only count non-duplicated columns
        idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
        # Reindex dataframe
        jobs_former_new = combined_jobs.reindex(idx)
        # Add to list of frames
        frame.append(jobs_former_new)
    
    #*******************************************************************************************************
    # PARTNER INVESTMENT JOB FILTER
    print('CaLcUlAtInG... PARTNER INVESTMENT JOB FILTER')
    
    for frame in lst_of_frames:
        # Are the partner investment jobs already in one of the jobs dataframes? If not, we should add them.
        # Create temporary dataframe and column to make checking the intersection between dataframes easier 
        # frame[4]: current jobs | frame[5]: former jobs | frame[6]: former new jobs
        jobs_combined = pd.concat([frame[4],frame[5],frame[6]])
        jobs_combined['person,company'] = jobs_combined['person_uuid'] + ',' + jobs_combined['org_uuid']
        # frame[3]: partner investments
        frame[3]['person,company'] = frame[3]['partner_uuid']+ ',' + frame[3]['investor_uuid']
        # Number of unique partner investments
        unique_PI = frame[3]['person,company'].unique()
        # Overlap between PI and combined J frames, create temporary jobs view
        # These PI are already found in J frames, so we do not need to include them
        jobs_already_in_J = jobs_combined[jobs_combined['person,company'].isin(unique_PI)] 
        # This will return non intersecting value of PI with temp J
        # These PI are not found in J, so we would like to include them
        PI_not_in_J = np.setdiff1d(unique_PI,jobs_already_in_J['person,company'].unique())
        # Need to create separate jobs dataframe for non intersecting PI/J person/company pairs
        grouped = frame[3][frame[3]['person,company'].isin(PI_not_in_J)].groupby(['partner_uuid','partner_name','investor_uuid','investor_name']).count()
        grouped_df = grouped.reset_index()[['partner_uuid','partner_name','investor_uuid','investor_name']]
        grouped_df['job_type'] = 'executive'
        # Add to list of frames
        frame.append(grouped_df)
    
    #*******************************************************************************************************
    # OTHER FIRM PARNTERS
    print('CaLcUlAtInG... OTHER FIRM PARTNER JOBS & INVESTMENTS FILTER')
    
    for frame in lst_of_frames:
        # OTHER FIRM PARNTERS - JOBS
        # Who are the other partners that work at the investment firms present in the neighborhood?
        # Get the unique investor uuids associated with the dataframes
        # frame[2]: from investments dataframe
        unique_investor_firm_A = list(frame[2]['investor_uuid'].unique())
        # frame[3]: from partner investments dataframe
        unique_investor_firm_B = list(frame[3]['investor_uuid'].unique())
        partners = list(frame[3]['partner_uuid'].unique())
        # Combine to get list of unique uuids of VC firms
        unique_firms = list(set(unique_investor_firm_A+unique_investor_firm_B))
        # Grab current jobs from Crunchbase for these investing firms
        # Exclude duplicate partner job (already represented by partners list calculated above)
        partner_jobs = CB_frames[4][(CB_frames[4]['org_uuid'].isin(unique_firms)) &  
                                    ~(CB_frames[4]['person_uuid'].isin(partners))].reset_index(drop=True)
        # Check they're not already in the current/former jobs dataframe
        # Combine into one temp data frame
        combined_jobs = pd.concat([frame[4], partner_jobs]).reset_index(drop=True) 
        df_gpby = combined_jobs.groupby(list(combined_jobs.columns))
        # Only count non-duplicated rows
        idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
        # Reindex dataframe
        partner_jobs = combined_jobs.reindex(idx)
        # Add to list of frames
        frame.append(partner_jobs)
        # OTHER FIRM PARNTERS - PARTNER INVESTMENTS
        # For these new partners, what companies are they invested in?
        # Get the unique parnter uuids associated with the dataframes
        other_partners = partner_jobs['person_uuid'].unique()
        other_partner_investments = CB_frames[3][CB_frames[3]['partner_uuid'].isin(other_partners)]
        # Check they're not already in the partner investments dataframe
        # Combine into one temp data frame
        combined_jobs = pd.concat([frame[3], other_partner_investments]).reset_index(drop=True) 
        df_gpby = combined_jobs.groupby(list(combined_jobs.columns))
        # Only count non-duplicated rows
        idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
        # Reindex dataframe
        other_partner_investments = combined_jobs.reindex(idx)
        # Add to list of frames
        frame.append(other_partner_investments)
    
    #*******************************************************************************************************
    # CURRENT OLD JOB FILTER
    print('CaLcUlAtInG... CURRENT OLD JOB FILTER')
    
    for frame in lst_of_frames:
        # Where did the current affiliated work previously?
        current_people = frame[4].person_uuid.unique() # Pull their IDs
        jobs_current_old = CB_frames[5][CB_frames[5].person_uuid.isin(current_people)] # Pull their current jobs from Crunchbase
        # Check they're not already in the current jobs dataframe
        # Combine into one temp data frame
        combined_jobs = pd.concat([frame[5], jobs_current_old]).reset_index(drop=True) 
        df_gpby = combined_jobs.groupby(list(combined_jobs.columns))
        # Only count non-duplicated columns
        idx = [x[0] for x in df_gpby.groups.values() if len(x) == 1]
        # Reindex dataframe
        jobs_current_old = combined_jobs.reindex(idx)
        # Add to list of frames
        frame.append(jobs_current_old)
        
    #*******************************************************************************************************
    # GET EXTRA ORG UUID ATTRIBUTES FROM INVESTMENTS & JOBS
    print('CaLcUlAtInG... EXTRA ORGANIZATION NODES')
    
    CB_orgs = pd.concat([CB_companies, CB_investors])
    for frame in lst_of_frames:
        unique_orgs = []
        # Investments
        unique_orgs.extend(list(frame[2]['investor_uuid'].unique()))
        # Partner investments
        unique_orgs.extend(list(frame[3]['investor_uuid'].unique()))
        # Former new jobs organizations
        unique_orgs.extend(list(frame[6]['org_uuid'].unique()))
        # Parter jobs organizations
        unique_orgs.extend(list(frame[7]['investor_uuid'].unique()))
        # Other parter investments organizations
        unique_orgs.extend(list(frame[9]['org_uuid'].unique()))
        # Current old jobs organizations
        unique_orgs.extend(list(frame[10]['org_uuid'].unique()))
        # Pull their organization information from Crunchbase
        new_org_nodes = CB_orgs[CB_orgs['uuid'].isin(list(set(unique_orgs)))]
        # Add to list of frames
        frame.append(new_org_nodes)
    
    #*******************************************************************************************************
    
    # Output print statements
    print('\nCrunchbase Neighborhood')
    print('NODES | OUTPUT FRAME 0/CB_companies {}'.format(CB_frames[0].shape))
    print('NODES | OUTPUT FRAME 1/CB_investors {}'.format(CB_frames[1].shape))
    print('NODES&EDGES | OUTPUT FRAME 2/CB_investments {}'.format(CB_frames[2].shape))
    print('NODES&EDGES | OUTPUT FRAME 3/CB_investment_partners {}'.format(CB_frames[3].shape))
    print('NODES&EDGES | OUTPUT FRAME 4/CB_jobs {}'.format(CB_frames[4].shape))
    print('NODES&EDGES | OUTPUT FRAME 5/CB_jobs_former {}'.format(CB_frames[5].shape))
    print('NODES&EDGES | OUTPUT FRAME 6/CB_jobs_former_new {}'.format(CB_frames[6].shape))
    print('NODES&EDGES | OUTPUT FRAME 7/CB_jobs_partner {}'.format(CB_frames[7].shape))
    print('NODES&EDGES | OUTPUT FRAME 8/CB_jobs_other_partners {}'.format(CB_frames[8].shape))
    print('NODES&EDGES | OUTPUT FRAME 9/CB_invest_other_partners {}'.format(CB_frames[9].shape))
    print('NODES&EDGES | OUTPUT FRAME 10/CB_jobs_current_old {}'.format(CB_frames[10].shape))
    print('NODES | OUTPUT FRAME 11/CB_extra_org_nodes {}'.format(CB_frames[11].shape))
    if model_uuids != []:
        print('\nModel Neighborhood')
        print('NODES | OUTPUT FRAME 0/model_companies {}'.format(model_frames[0].shape))
        print('NODES | OUTPUT FRAME 1/model_investors {}'.format(model_frames[1].shape))
        print('NODES&EDGES | OUTPUT FRAME 2/model_investments {}'.format(model_frames[2].shape))
        print('NODES&EDGES | OUTPUT FRAME 3/model_investment_partners {}'.format(model_frames[3].shape))
        print('NODES&EDGES | OUTPUT FRAME 4/model_jobs {}'.format(model_frames[4].shape))
        print('NODES&EDGES | OUTPUT FRAME 5/model_jobs_former {}'.format(model_frames[5].shape))
        print('NODES&EDGES | OUTPUT FRAME 6/model_jobs_former_new {}'.format(model_frames[6].shape))
        print('NODES&EDGES | OUTPUT FRAME 7/model_jobs_partner {}'.format(model_frames[7].shape))
        print('NODES&EDGES | OUTPUT FRAME 8/model_jobs_other_partners {}'.format(model_frames[8].shape))
        print('NODES&EDGES | OUTPUT FRAME 9/model_invest_other_partners {}'.format(model_frames[9].shape))
        print('NODES&EDGES | OUTPUT FRAME 10/model_jobs_current_old {}'.format(model_frames[10].shape))
        print('NODES | OUTPUT FRAME 11/model_extra_org_nodes {}'.format(model_frames[11].shape))
        return lst_of_frames
    print('\nPledge 1% Neighborhood')
    print('NODES | OUTPUT FRAME 0/P1_companies {}'.format(P1_frames[0].shape))
    print('NODES | OUTPUT FRAME 1/P1_investors {}'.format(P1_frames[1].shape))
    print('NODES&EDGES | OUTPUT FRAME 2/P1_investments {}'.format(P1_frames[2].shape))
    print('NODES&EDGES | OUTPUT FRAME 3/P1_investment_partners {}'.format(P1_frames[3].shape))
    print('NODES&EDGES | OUTPUT FRAME 4/P1_jobs {}'.format(P1_frames[4].shape))
    print('NODES&EDGES | OUTPUT FRAME 5/P1_jobs_former {}'.format(P1_frames[5].shape))
    print('NODES&EDGES | OUTPUT FRAME 6/P1_jobs_former_new {}'.format(P1_frames[6].shape))
    print('NODES&EDGES | OUTPUT FRAME 7/P1_jobs_partner {}'.format(P1_frames[7].shape))
    print('NODES&EDGES | OUTPUT FRAME 8/P1_jobs_other_partners {}'.format(P1_frames[8].shape))
    print('NODES&EDGES | OUTPUT FRAME 9/P1_invest_other_partners {}'.format(P1_frames[9].shape))
    print('NODES&EDGES | OUTPUT FRAME 10/P1_jobs_current_old {}'.format(P1_frames[10].shape))
    print('NODES | OUTPUT FRAME 11/P1_extra_org_nodes {}'.format(P1_frames[11].shape))
    # Skip Not P1 Calculations
    if skip_not_p1 is False:
        print('\n~Pledge 1% Neighborhood')
        print('NODES | OUTPUT FRAME 0/not_P1_companies {}'.format(not_P1_frames[0].shape))
        print('NODES | OUTPUT FRAME 1/not_P1_investors {}'.format(not_P1_frames[1].shape))
        print('NODES&EDGES | OUTPUT FRAME 2/not_P1_investments {}'.format(not_P1_frames[2].shape))
        print('NODES&EDGES | OUTPUT FRAME 3/not_P1_investment_partners {}'.format(not_P1_frames[3].shape))
        print('NODES&EDGES | OUTPUT FRAME 4/not_P1_jobs {}'.format(not_P1_frames[4].shape))
        print('NODES&EDGES | OUTPUT FRAME 5/not_P1_jobs_former {}'.format(not_P1_frames[5].shape))
        print('NODES&EDGES | OUTPUT FRAME 6/not_P1_jobs_former_new {}'.format(not_P1_frames[6].shape))
        print('NODES&EDGES | OUTPUT FRAME 7/not_P1_jobs_partner {}'.format(not_P1_frames[7].shape))
        print('NODES&EDGES | OUTPUT FRAME 8/not_P1_jobs_other_partners {}'.format(not_P1_frames[8].shape))
        print('NODES&EDGES | OUTPUT FRAME 9/not_P1_invest_other_partners {}'.format(not_P1_frames[9].shape))
        print('NODES&EDGES | OUTPUT FRAME 10/not_P1_jobs_current_old {}'.format(not_P1_frames[10].shape))
        print('NODES | OUTPUT FRAME 11/not_P1_extra_org_nodes {}'.format(not_P1_frames[11].shape))
    return lst_of_frames

def load_vertices(sframes, g):
    # For jobs dataframes
    for idx in [4,5,6,8,10]:
        # Keep relevant node attributes
        frame_temp = sframes[idx][['person_uuid', 'person_name']].rename({'person_uuid':'__id', 'person_name':'name'})
        frame_temp['__node_type'] = 'person'
        # Add p1_tag to the vertex
        frame_temp['p1_tag'] = 0
        g = g.add_vertices(vertices=frame_temp, vid_field='__id')
    # For jobs and partner investments dataframes
    for idx in [2,3,4,5,6,8,9,10]:
        # Keep relevant node attributes
        frame_temp = sframes[idx][['org_uuid', 'org_name', 'p1_tag']].rename({'org_uuid':'__id', 'org_name':'name'})
        frame_temp['__node_type'] = 'company'
        # Add p1_tag to the vertex
        frame_temp['p1_tag'] = frame_temp['p1_tag'].apply(lambda x: 0 if (x=="" or x==0) else 1)
        frame_temp['p1_tag'] = frame_temp['p1_tag'].astype(int)
        g = g.add_vertices(vertices=frame_temp, vid_field='__id')
    # For investments dataframes
    for idx in [2,3,7,9]:
        # Keep relevant node attributes
        frame_temp = sframes[idx][['investor_uuid', 'investor_name']].rename({'investor_uuid':'__id', 'investor_name':'name'})
        frame_temp['__node_type'] = 'investor'
        # Add p1_tag to the vertex
        frame_temp['p1_tag'] = 0
        g = g.add_vertices(vertices=frame_temp, vid_field='__id')
    # For partner investments dataframes
    for idx in [3,7,9]:
        # Keep relevant node attributes
        frame_temp = sframes[idx][['partner_uuid', 'partner_name']].rename({'partner_uuid':'__id', 'partner_name':'name'})
        frame_temp['__node_type'] = 'person'
        # Add p1_tag to the vertex
        frame_temp['p1_tag'] = 0
        g = g.add_vertices(vertices=frame_temp, vid_field='__id')
    # Organizations
    for idx in [0,1,11]:
        # Keep relevant node attributes
        frame_temp = sframes[idx][['uuid', 'name', 'primary_role', 'p1_tag']].rename({'uuid':'__id', 'primary_role':'__node_type'})
        # Add p1_tag to the vertex
        frame_temp['p1_tag'] = frame_temp['p1_tag'].apply(lambda x: 0 if (x=="" or x==0) else 1)
        frame_temp['p1_tag'] = frame_temp['p1_tag'].astype(int)
        # Load into graph
        g = g.add_vertices(vertices=frame_temp, vid_field='__id')
    # Return SGraph
    return g

def find_p1_affiliations(p1_sframes):
    frames = p1_sframes.copy()
    # Combine company and investor Pledge 1% dataframes, keeping only the uuid column
    p1_affiliations = frames[0][['uuid']].append(frames[1][['uuid']])
    # Add edge connecting to Pledge 1% uuid
    p1_affiliations['p1_uuid'] = 'fd9e2d10-a882-c6f4-737e-fd388d4ffd7c'
    # Create id, source, destination fields in SFrame
    p1_affiliations = p1_affiliations.rename({'uuid':'src','p1_uuid':'dst'})
    p1_affiliations['p1_tag'] = 1
    # Return SFrame
    return p1_affiliations

def load_edges(sframes, g, p1_affiliations=[], include_edges=[2,3], reverse=False, add_weights=False):
    w = {'status':{'primary':3,'secondary':2,'tertiary':1}, '__edge_type':{'job':1, 'investment':2}}
    # Since it is a directed graph, need to include option for reverse direction
    # Forward
    source = 'src'
    destination = 'dst'
    # Reverse
    if reverse:
        source = 'dst'
        destination = 'src'
    if type(p1_affiliations) == SFrame:
        # P1 Companies: Company/Investor --> Pledge 1%
        g = g.add_edges(edges=p1_affiliations, src_field=source, dst_field=destination)
        if add_weights:
            frame_temp['weight'] = 6
    # Investments: Investor --> Company
    # Create id, source, destination fields in SFrame
    frame_temp = sframes[2][['investment_uuid','investor_uuid','org_uuid','investment_type','raised_amount_usd','investor_count','is_lead_investor','lead_investor_count']].rename({'investment_uuid':'__id','investor_uuid':'src','org_uuid':'dst'})
    frame_temp['__edge_type'] = 'investment'
    frame_temp['status'] = 'primary'
    if add_weights:
        frame_temp['weight'] = w['__edge_type']['investment'] * w['status']['primary']
    g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
    # Partner Investments, Investments: Person --> Company
    # Create id, source, destination fields in SFrame
    frame_temp = sframes[3][['investment_uuid','partner_uuid','org_uuid','investment_type','raised_amount_usd','investor_count']].rename({'investment_uuid':'__id','partner_uuid':'src','org_uuid':'dst'})
    frame_temp['__edge_type'] = 'investment'
    frame_temp['status'] = 'primary'
    if add_weights:
        frame_temp['weight'] = w['__edge_type']['investment'] * w['status']['primary']
    g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
    # Partner Investments, Investments: Investor --> Company
    # Create id, source, destination fields in SFrame
    frame_temp = sframes[3][['investor_uuid','org_uuid','investment_type','investor_count']].rename({'investor_uuid':'src','org_uuid':'dst'})
    frame_temp['__edge_type'] = 'investment'
    frame_temp['status'] = 'secondary'
    if add_weights:
        frame_temp['weight'] = w['__edge_type']['investment'] * w['status']['secondary']
    # Secondary relationships, skip if not specified at input
    if 2 in include_edges:
        g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
    # Partner Investments, Jobs: Person --> Company
    # Create id, source, destination fields in SFrame
    frame_temp = sframes[7][['partner_uuid','investor_uuid']].rename({'partner_uuid':'src','investor_uuid':'dst'})
    frame_temp['__edge_type'] = 'job'
    frame_temp['status'] = 'secondary'
    if add_weights:
        frame_temp['weight'] = w['__edge_type']['job'] * w['status']['secondary']
    # Secondary relationships, skip if not specified at input
    if 2 in include_edges:
        g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)    
    # Other Partner Investments, Investments: Person --> Company
    # Create id, source, destination fields in SFrame
    frame_temp = sframes[9][['investment_uuid','partner_uuid','org_uuid','investment_type','raised_amount_usd','investor_count']].rename({'investment_uuid':'__id','partner_uuid':'src','org_uuid':'dst'})
    frame_temp['__edge_type'] = 'investment'
    frame_temp['status'] = 'tertiary'
    if add_weights:
        frame_temp['weight'] = w['status']['tertiary'] * w['__edge_type']['investment']
    # Tertiary relationships, skip if not specified at input
    if 3 in include_edges:
        g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
    # Jobs: Person --> Company
    for idx in [4,5,6,8,10]:
        # Create id, source, destination fields in SFrame
        frame_temp = sframes[idx][['job_uuid','person_uuid','org_uuid','job_type','title']].rename({'job_uuid':'__id','person_uuid':'src','org_uuid':'dst'})
        frame_temp['__edge_type'] = 'job'
        # Current jobs
        if idx == 4:
            frame_temp['status'] = 'primary'
            if add_weights:
                frame_temp['weight'] = w['status']['primary'] * w['__edge_type']['job']
            g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
            continue
        # Secondary relationships, skip if not specified at input
        if 2 in include_edges:
            # Former jobs | Former new jobs | Current old jobs 
            if idx in [5,6,10]:
                frame_temp['status'] = 'secondary'
                if add_weights:
                    frame_temp['weight'] = w['status']['secondary'] * w['__edge_type']['job']
                g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
                continue  
        # Tertiary relationships, skip if not specified at input
        if 3 in include_edges:
            # Other partners at firm
            if idx == 8:
                frame_temp['status'] = 'tertiary'
                if add_weights:
                    frame_temp['weight'] = w['status']['tertiary'] * w['__edge_type']['job']
                g = g.add_edges(edges=frame_temp, src_field=source, dst_field=destination)
                continue
    # Return SGraph
    return g

def update_cb_weights(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        edge['weight'] = 0
        edge['weight_status'] = 0
        edge['weight_type'] = 0
        if edge['status'] == 'primary':
            edge['weight_status'] = 3
        if edge['status'] == 'secondary':
            edge['weight_status'] = 2
        if edge['status'] == 'tertiary':
            edge['weight_status'] = 1
        if edge['__edge_type'] == 'job':
            edge['weight_type'] = 1
        if edge['__edge_type'] == 'investment':
            edge['weight_type'] = 2
        edge['weight'] = edge['weight_status'] * edge['weight_type']
    return (src, edge, dst)

def update_pagerank_weight(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        dst['pagerank'] += src['prev_pagerank'] * edge['weight']
    return (src, edge, dst)

def update_pagerank_reset_prob(src, edge, dst):
    global reset
    if src['__id'] != dst['__id']: # ignore self-links
        dst['pagerank'] *= (1 - reset)
        dst['pagerank'] += reset
    return (src, edge, dst)

def update_pagerank_prev_to_current(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        src['prev_pagerank'] = src['pagerank']
    return (src, edge, dst)

def sum_weight(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        src['total_weight'] += edge['weight']
    return src, edge, dst

def make_pagerank_zero(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        dst['pagerank'] = 0
    return src, edge, dst

def update_l1_delta(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        dst['l1_delta'] = abs(dst['pagerank'] - dst['prev_pagerank'])
        src['l1_delta'] = abs(src['pagerank'] - src['prev_pagerank'])
    return src, edge, dst

def normalize_weight(src, edge, dst):
    if src['__id'] != dst['__id']: # ignore self-links
        edge['weight'] /= src['total_weight']
    return src, edge, dst

def pagerank_weighted(input_graph, reset_prob=0.15, threshold=0.01, max_iterations=3):
    g = SGraph(input_graph.vertices, input_graph.edges)
    global reset
    reset = reset_prob
    # compute normalized edge weight
    g.vertices['total_weight'] = 0.0
    g = g.triple_apply(sum_weight, ['total_weight'])
    g = g.triple_apply(normalize_weight, ['weight'])
    del g.vertices['total_weight']
    # initialize vertex field
    g.vertices['prev_pagerank'] = 1.0
    it = 0
    total_l1_delta = len(g.vertices)
    start = time.time()
    while(total_l1_delta > threshold and it < max_iterations):
        if 'pagerank' not in g.get_vertex_fields():
            g.vertices['pagerank'] = 0.0
        else:
            g = g.triple_apply(make_pagerank_zero, ['pagerank'])
        g = g.triple_apply(update_pagerank_weight, ['pagerank'])
        g = g.triple_apply(update_pagerank_reset_prob, ['pagerank'])
        if 'l1_delta' not in g.get_vertex_fields():
            g.vertices['l1_delta'] = (g.vertices['pagerank'] - g.vertices['prev_pagerank']).apply(lambda x: abs(x))
        else:
            g = g.triple_apply(update_l1_delta, ['l1_delta'])
        total_l1_delta = g.vertices['l1_delta'].sum()
        g = g.triple_apply(update_pagerank_prev_to_current, ['prev_pagerank'])
        print ("Iteration %d: total pagerank changed in L1 = %f" % (it, total_l1_delta))
        it = it + 1
    print ("Weighted pagerank finished in: %f secs" % (time.time() - start))
    del g.vertices['prev_pagerank']
    return g.vertices

### Add `w_spath`

In [ ]:
# wspath = pd.read_csv('wspath.csv')
# wspath['__id'] = wspath['uuid']
# wspath = wspath.drop('uuid', axis=1)
# wspath.to_csv('wspath.csv', index=False)
# wspath = pd.read_csv('wspath.csv')
# wspath.head()

In [ ]:
# spath = pd.read_csv('wspath.csv')

# for neighborhood in ['Model_DF_D4', 'Model_DF_D5']: # 2 times
#     for scenario in ['G', 'GB', 'GBR']: # 3 times
#         for idx in range(1,10): # 10 times
#             print('{} | {} | {}'.format(neighborhood,scenario,idx))
#             path = 'files/output/{}/{}/{}.csv'.format(neighborhood,scenario,idx)
#             DF = pd.read_csv(path)
#             DF_MERGE = pd.merge(DF, wspath, on='__id', how='inner')
#             DF_MERGE.to_csv(path, index=False)

### Add `spath`

In [13]:
# spath = pd.read_csv('spath.csv')

# for neighborhood in ['Model_DF_D4', 'Model_DF_D5']: # 2 times
#     for scenario in ['G', 'GB', 'GBR']: # 3 times
#         for idx in range(1,10): # 10 times
#             print('{} | {} | {}'.format(neighborhood,scenario,idx))
#             path = 'files/output/{}/{}/{}.csv'.format(neighborhood,scenario,idx)
#             DF = pd.read_csv(path)
#             DF_MERGE = pd.merge(DF, spath, on='__id', how='inner')
#             DF_MERGE.to_csv(path, index=False)

Model_DF_D4 | G | 1
Model_DF_D4 | G | 2
Model_DF_D4 | G | 3
Model_DF_D4 | G | 4
Model_DF_D4 | G | 5
Model_DF_D4 | G | 6
Model_DF_D4 | G | 7
Model_DF_D4 | G | 8
Model_DF_D4 | G | 9
Model_DF_D4 | GB | 1
Model_DF_D4 | GB | 2
Model_DF_D4 | GB | 3
Model_DF_D4 | GB | 4
Model_DF_D4 | GB | 5
Model_DF_D4 | GB | 6
Model_DF_D4 | GB | 7
Model_DF_D4 | GB | 8
Model_DF_D4 | GB | 9
Model_DF_D4 | GBR | 1
Model_DF_D4 | GBR | 2
Model_DF_D4 | GBR | 3
Model_DF_D4 | GBR | 4
Model_DF_D4 | GBR | 5
Model_DF_D4 | GBR | 6
Model_DF_D4 | GBR | 7
Model_DF_D4 | GBR | 8
Model_DF_D4 | GBR | 9
Model_DF_D5 | G | 1
Model_DF_D5 | G | 2
Model_DF_D5 | G | 3
Model_DF_D5 | G | 4
Model_DF_D5 | G | 5
Model_DF_D5 | G | 6
Model_DF_D5 | G | 7
Model_DF_D5 | G | 8
Model_DF_D5 | G | 9
Model_DF_D5 | GB | 1
Model_DF_D5 | GB | 2
Model_DF_D5 | GB | 3
Model_DF_D5 | GB | 4
Model_DF_D5 | GB | 5
Model_DF_D5 | GB | 6
Model_DF_D5 | GB | 7
Model_DF_D5 | GB | 8
Model_DF_D5 | GB | 9
Model_DF_D5 | GBR | 1
Model_DF_D5 | GBR | 2
Model_DF_D5 | GBR | 

### Update `__id` to `uuid`

In [ ]:
for neighborhood in ['Model_DF_D4', 'Model_DF_D5']: # 2 times
    for scenario in ['G', 'GB', 'GBR']: # 3 times
        for idx in range(1): # 10 times
            print('{} | {} | {}'.format(neighborhood,scenario,idx))
            path = 'files/output/{}/{}/{}.csv'.format(neighborhood,scenario,idx)
            DF = pd.read_csv(path)
            DF['uuid'] = DF['__id']
            DF = DF.drop('__id', axis=1)
            DF_MERGE.to_csv(path, index=False)

In [15]:
DF_MERGE.columns.to_list()

['__id',
 'pr_0',
 'w_pr_0',
 'in_deg_0',
 'out_deg_0',
 'tri_0',
 'pr_1',
 'w_pr_1',
 'in_deg_1',
 'out_deg_1',
 'pr_2',
 'w_pr_2',
 'kc_2',
 'in_deg_2',
 'out_deg_2',
 'tri_2',
 'spath_top_3_0',
 'spath_top_3_1',
 'spath_top_3_2',
 'spath_top_3_3',
 'spath_top_3_4',
 'spath_top_min_3',
 'w_spath_top_3_0',
 'w_spath_top_3_1',
 'w_spath_top_3_2',
 'w_spath_top_3_3',
 'w_spath_top_3_4',
 'w_spath_top_min_3',
 'kc_3',
 'in_deg_3',
 'out_deg_3',
 'spath']